In [1]:
#Import Packages
import pandas as pd
import numpy as np
import os
import itertools
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import datetime
import matplotlib
from herbie import Herbie
import pickle
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import pysolar.solar as solar
from geographiclib.geodesic import Geodesic
import xarray as xr
import pytz
import simplekml
import cartopy
import sys
sys.path.append('..')
import funcs.ac_funcs as ac
import funcs.plotting as plotting


pd.options.mode.chained_assignment = None


In [24]:
def add_oof_trace(fig,oof_df,spec,color,name,errors=False):
    if errors:
        fig.add_trace(go.Scatter(
            x = oof_df.index,
            y = oof_df[spec],
            name=name,
            error_y = dict(
                type='data', # value of error bar given in data coordinates
                array=oof_df[f'{spec}_error'],
                visible=True,
                color = 'grey',
                width = 0.1),
            mode='markers',
            marker={'color':color,'size':2},
        ))
    else:
        fig.add_trace(go.Scatter(
            x = oof_df.index,
            y = oof_df[spec],
            name=name,
            mode='markers',
            marker={'color':color,'size':2},
        ))

In [20]:
existing_results_path = '/uufs/chpc.utah.edu/common/home/lin-group15/agm/em27/ua/results/'
oof_destination_path = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/EM27_oof/ua/'

ac.copy_em27_oofs_to_singlefolder(existing_results_path,oof_destination_path)

Copying Files


In [21]:
#base_project_dir = '/Users/agmeyer4/LAIR_1/'
base_project_dir = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/'

inst_ids = ['ha','ua']
flag = 0
resample = None

#define datetime range and interval parameters
dt1_str = '2023-07-07 00:00:00'#start datetime
dt2_str = '2023-07-12 23:59:59' #end datetime
timezone = 'UTC' #this should nearly always be UTC, but can identify if your datetime strings above are in a different timezone
dt1 = ac.dtstr_to_dttz(dt1_str,timezone)
dt2 = ac.dtstr_to_dttz(dt2_str,timezone)

my_oof_managers = {}
oof_dfs = {}
for inst_id in inst_ids:
    data_folder = os.path.join(base_project_dir,f'Data/EM27_oof/{inst_id}')
    my_oof_managers[inst_id] = ac.oof_manager(data_folder,timezone)
    df = my_oof_managers[inst_id].load_oof_df_inrange(dt1,dt2)
    if flag is not None:
        df = df.loc[df['flag']==flag]
    if resample is not None:
        df = df.resample(resample).mean()
    oof_dfs[inst_id] = df

In [25]:
# xlims= [datetime.datetime.combine(em27_resampled.index[0].date(),datetime.time(hour=6)),
#         datetime.datetime.combine(em27_resampled.index[-1].date(),datetime.time(hour=22))]
spec = 'xch4(ppm)'
fig = go.Figure()
inst_id = 'ha'
add_oof_trace(fig,oof_dfs[inst_id],'xch4(ppm)','red',inst_id)
inst_id = 'ua'
add_oof_trace(fig,oof_dfs[inst_id],'xch4(ppm)','blue',inst_id)
fig.update_yaxes(title_text = spec)
fig.update_layout(
    height=400,
)
fig.show()